In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv2DTranspose, BatchNormalization, Flatten, Reshape, LeakyReLU, Activation
from keras.activations import sigmoid
import tensorflow as tf

from utilities import show_graph

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

Using TensorFlow backend.


In [2]:
mnist = input_data.read_data_sets('./data', one_hot=False)
image_size = (28, 28, 1)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [3]:
# tf.reset_default_graph()
def discriminator(print_summary=False):
    with tf.variable_scope('discriminator'):
        discriminator = Sequential()
        discriminator.add(Conv2D(32, (5, 5), strides=2, activation=LeakyReLU(),
                         padding='same', input_shape=image_size))
        discriminator.add(Conv2D(64, (5, 5), strides=2,
                         padding='same', input_shape=image_size))
        discriminator.add(BatchNormalization())
        discriminator.add(LeakyReLU())
        discriminator.add(Flatten())
        discriminator.add(Dense(1, activation='sigmoid'))
    
    if print_summary:
        print(discriminator.summary())
        
    return discriminator

In [4]:
def generator(print_summary=False):
    z_dim = 100
    depth = 128
    with tf.variable_scope('generator'):
        generator = Sequential()
        generator.add(Dense(7*7*depth, input_dim=z_dim))
        generator.add(Reshape((7,7,depth)))
        generator.add(BatchNormalization())
        generator.add(LeakyReLU())

        depth = int(depth/2)
        generator.add(Conv2DTranspose(depth, 5, padding='same', strides=2))
        generator.add(BatchNormalization())
        generator.add(LeakyReLU())

        depth = int(depth/2)
        generator.add(Conv2DTranspose(depth, 5, padding='same', strides=1))
        generator.add(BatchNormalization())
        generator.add(LeakyReLU())

        depth = int(depth/2)
        generator.add(Conv2DTranspose(depth, 5, padding='same', strides=1))
        generator.add(BatchNormalization())
        generator.add(LeakyReLU())

        # depth = int(depth/2)
        generator.add(Conv2DTranspose(1, 5, padding='same', strides=2))
        generator.add(BatchNormalization())
        generator.add(Activation(sigmoid))
    
    if print_summary:
        print(generator.summary())
        
    return generator

In [5]:
def adversarial_model(generator, discriminator, print_summary=False):
    adversarial_model = Sequential()
    adversarial_model.add(generator)
    discriminator.trainable = False
    adversarial_model.add(discriminator)
    if print_summary:
        adversarial_model.summary()
    return adversarial_model

In [6]:
def mnist_gen(batch_size):
    while 1:
        data = mnist.train.next_batch(batch_size)
        yield data[0].reshape((-1,)+image_size)

In [7]:
batch_size = 128
z_dim = 100
mnist_data = mnist_gen(batch_size)

d = discriminator()
g = generator()
d_on_g = adversarial_model(g, d)
# d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
# g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
g.compile(loss='binary_crossentropy', optimizer="adam")
d_on_g.compile(loss='binary_crossentropy', optimizer="adam")
d.trainable = True
d.compile(loss='binary_crossentropy', optimizer="adam")
# for epoch in range(100):
#     print("Epoch is", epoch)
#     print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
#     for index in range(int(X_train.shape[0]/BATCH_SIZE)):
image_batch =  next(mnist_data)
noise = np.random.randn(batch_size, z_dim)

#         noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
#         image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
generated_images = g.predict(noise, verbose=0)
#         if index % 20 == 0:
#             image = combine_images(generated_images)
#             image = image*127.5+127.5
#             Image.fromarray(image.astype(np.uint8)).save(
#                 str(epoch)+"_"+str(index)+".png")
X = np.concatenate((image_batch, generated_images))
y = [1] * batch_size + [0] * batch_size
d_loss = d.train_on_batch(X, y)

InvalidArgumentError: You must feed a value for placeholder tensor 'generator/dense_2_input' with dtype float
	 [[Node: generator/dense_2_input = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'generator/dense_2_input', defined at:
  File "/root/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/root/miniconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super

In [ ]:
image_batch.shape

In [ ]:
g

In [ ]:
d

In [ ]:
type(d)

In [ ]:
generator.compile(loss='binary_crossentropy', optimizer='adam')
adversarial_model.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer='adam') 

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(discriminator).create(prog='dot', format='svg'))

In [ ]:
batch_size = 128
mnist_data = mnist_gen(batch_size)
# for i in range(1000):
z = np.random.randn(batch_size, z_dim)
gen_im = generator.predict(z)
mnist_batch = next(mnist_data)
x = np.vstack([mnist_batch, gen_im])
y = np.ones((2*batch_size, 1), dtype='float32')
y[batch_size:] = 0
discriminator.trainable = True
discriminator.train_on_batch(x, y)

# y = np.ones((batch_size, 1))
# adversarial_model.train_on_batch(z, y)

In [ ]:
generator.trainable